## Script to get clean feature table vefore passing to model

Cleaned feature table will be saved as 'clean_ft_table.csv'

- Handle missing values (either by removing the feature or removing the sample)

- Remove 'mistakes' (values outside the human range)

- Remove patients who were transferred from surgery (cases usually resolved quickly and predictably)

#### import libraries

In [1]:
import sys
sys.path.append("../extumate")


import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from extumate.config import data_dir, processed_dir, features_dir

#### User selects first feature to start dataframe

In [3]:
base_feature = 'gender'
export_table_name = 'clean_ft_table_labs_strip'

#### get feature library

In [4]:
df = pd.read_feather(processed_dir+base_feature)

In [5]:
df_master = df.copy()

In [6]:
# select only useful columns for master dataframe
df_master=df_master[['hadm_id', 'endtime', 'time_on_vent', 're_intub_class',
                     'gender','admission_type','anchor_age']]

#### load each feature and merge with df_master on hadm_id

In [7]:
# all feature names are the same as the feature column so we can find the name of all features in
# the 'selected_features' directory, load each feature in turn, select only the relevant columns and merge these 
# with df_master on 'hadm_id'
dirList= os.listdir(features_dir)
filename = []

for fname in dirList:
    feature = np.append(filename, fname)
    feature_table = pd.read_feather('../data/processed/selected_features/'+fname)
    feature_columns = feature_table[['hadm_id',fname]]
    df_master = pd.merge(left = df_master, right = feature_columns, 
                     how='left', left_on = 'hadm_id',right_on = 'hadm_id')

In [8]:
df_master

,hadm_id,endtime,time_on_vent,re_intub_class,gender,admission_type,anchor_age,spontrr,heartrate,std_spontrr,...,height,lactic_acid,bnp,hemoglobin,wbg,tidalvolume,temp,std_bloodpressure,pulseox,std_temp
0,28038802,2185-12-20 09:59:00,108.800000,0,M,EW EMER.,71,17.0,87.0,5.560918,...,NaN,0.7,NaN,12.6,6.6,568.0,36.555556,10.830303,88.0,0.274986
1,21790335,2140-03-11 13:44:00,33.000000,0,F,URGENT,34,13.0,78.0,0.447214,...,NaN,2.9,NaN,9.1,10.5,578.0,36.888889,1.923538,92.0,NaN
2,24357541,2177-02-10 14:00:00,659.416667,0,F,URGENT,70,0.0,127.0,11.631239,...,59.027559,0.8,48581.0,8.5,8.4,0.0,37.500000,7.120393,94.0,NaN
3,22188993,2145-11-04 18:40:00,143.666667,0,M,EW EMER.,68,27.0,95.0,0.726483,...,74.007874,1.8,NaN,11.0,5.4,594.0,37.388889,3.669696,98.0,NaN
4,21880799,2134-05-22 17:58:00,54.966667,0,M,EW EMER.,52,24.0,100.0,9.227289,...,66.070866,1.0,NaN,7.8,8.4,500.0,36.555556,NaN,100.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16399,29960248,2146-12-09 14:23:00,73.466667,1,F,DIRECT EMER.,65,13.0,94.0,1.863525,...,61.011811,1.0,NaN,10.3,18.6,519.0,37.055556,3.761299,96.0,NaN
16400,29962016,2135-10-30 11:45:00,218.083333,1,F,EW EMER.,53,17.0,80.0,1.772811,...,64.086614,1.4,NaN,9.0,10.7,308.0,36.888889,12.378938,100.0,NaN
16401,29974575,2131-03-03 19:54:00,98.900000,1,M,EW EMER.,72,21.0,71.0,8.952254,...,70.039370,0.8,NaN,10.0,6.0,447.0,36.888889,296.433466,100.0,NaN
16402,29987115,2148-02-19 10:00:00,44.000000,1,F,URGENT,43,12.0,88.0,4.527693,...,59.921260,1.1,NaN,9.0,6.5,594.0,36.722222,7.949843,99.0,NaN


#### select only patients that were intubated for longer than 6 hours 

To get rid of all the following warnings, sHould change this to the df.where df.copy together? https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#the-where-method-and-masking

In [9]:
df6 = df_master.copy()
#df6 = df_master[(df_master['time_on_vent']>6)]
df6 = df_master[['hadm_id', 'endtime', 'time_on_vent', 're_intub_class', 'gender',
       'admission_type', 'anchor_age', 'heartrate',
       'weight',  'hco3', 'pulseox',
        
       'creatinine', 'bun',  'height',
       'tidalvolume', 'temp',
       ]]

In [10]:
df6.columns

Index(['hadm_id', 'endtime', 'time_on_vent', 're_intub_class', 'gender',
       'admission_type', 'anchor_age', 'heartrate', 'weight', 'hco3',
       'pulseox', 'creatinine', 'bun', 'height', 'tidalvolume', 'temp'],
      dtype='object')

In [11]:
df6.isnull().sum()

hadm_id              0
endtime              0
time_on_vent         0
re_intub_class       0
gender               0
admission_type       0
anchor_age           0
heartrate            1
weight            2949
hco3               165
pulseox              1
creatinine         162
bun                162
height            4108
tidalvolume         31
temp              2081
dtype: int64

In [12]:
df6.dropna(subset = ['height'],inplace=True)
df6.dropna(subset = ['weight'],inplace=True)
df6.dropna(subset = ['temp'],inplace=True)
df6.dropna(subset = ['hco3'],inplace=True)
df6.dropna(subset = ['creatinine'],inplace=True)
df6.dropna(subset = ['bun'],inplace=True)
df6.dropna(subset = ['tidalvolume'],inplace=True)

<ipython-input-12-a18eb4ead1e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6.dropna(subset = ['height'],inplace=True)
<ipython-input-12-a18eb4ead1e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6.dropna(subset = ['weight'],inplace=True)
<ipython-input-12-a18eb4ead1e8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6.dropna(subset = ['temp'],inplace=True)
<ipython-input-12-a18eb4ead1e8>:4: SettingWithCopyWarning: 
A v

In [14]:
df6[df6['re_intub_class']==0]

,hadm_id,endtime,time_on_vent,re_intub_class,gender,admission_type,anchor_age,heartrate,weight,hco3,pulseox,creatinine,bun,height,tidalvolume,temp
15197,20024229,2132-08-28 17:30:00,25.500000,1,M,OBSERVATION ADMIT,61,81.0,196.7,31.0,95.0,0.6,17.0,70.039370,390.0,36.888889
15198,20025172,2140-01-04 11:09:00,93.116667,1,F,OBSERVATION ADMIT,58,121.0,106.0,24.0,90.0,0.6,21.0,55.952756,384.0,37.166667
15199,20034762,2164-06-08 14:00:00,29.833333,1,F,URGENT,85,81.0,166.1,25.0,99.0,0.7,15.0,61.011811,432.0,36.222222
15200,20035700,2123-02-05 14:52:00,100.500000,1,M,OBSERVATION ADMIT,42,101.0,163.9,27.0,98.0,0.8,9.0,68.055118,538.0,37.666667
15203,20050336,2170-07-15 16:00:00,157.000000,1,F,EW EMER.,76,110.0,118.8,48.0,90.0,0.6,31.0,66.070866,292.0,37.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16397,29947543,2144-04-02 15:17:00,134.333333,1,F,EW EMER.,68,108.0,77.4,26.0,96.0,0.5,13.0,59.921260,320.0,37.611111
16398,29951431,2173-10-10 09:32:00,14.783333,1,M,EW EMER.,65,75.0,189.2,24.0,100.0,0.9,18.0,70.039370,498.0,37.722222
16399,29960248,2146-12-09 14:23:00,73.466667,1,F,DIRECT EMER.,65,94.0,139.0,21.0,96.0,2.9,62.0,61.011811,519.0,37.055556
16400,29962016,2135-10-30 11:45:00,218.083333,1,F,EW EMER.,53,80.0,160.6,34.0,100.0,0.5,13.0,64.086614,308.0,36.888889


In [ ]:
df6.describe()

## Exclusion Criteria

#### Tidal volume should be below 2000, weight should be above 80. Remove samples where this is not the case

In [ ]:
df6.drop(df6[(df6['tidalvolume'] > 2000)|(df6['tidalvolume'] < 100)].index, inplace = True) 

In [ ]:
df6.drop(df6[df6['weight'] < 50].index, inplace = True)

#### Remove surgical same-day admissions (since it's usually straightforward when these patients can be extubated)

In [15]:
df6['admission_type'].value_counts()

EW EMER.                       4448
URGENT                         2143
OBSERVATION ADMIT               892
SURGICAL SAME DAY ADMISSION     827
DIRECT EMER.                    364
ELECTIVE                        293
EU OBSERVATION                    2
DIRECT OBSERVATION                1
Name: admission_type, dtype: int64

In [16]:
df6.drop(df6.loc[df6['admission_type']=='DIRECT OBSERVATION'].index, inplace=True)
df6.drop(df6.loc[df6['admission_type']=='EU OBSERVATION'].index, inplace=True)
df6.drop(df6.loc[df6['admission_type']=='SURGICAL SAME DAY ADMISSION'].index, inplace=True)

/home/anne_717/anaconda3/envs/extumate/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
df6.reset_index(inplace=True)

In [ ]:
df6.drop('index',axis=1,inplace=True)

In [ ]:
df6

In [ ]:
df6.to_feather(processed_dir+export_table_name)